In [4]:
from string import punctuation
import re

In [5]:
# opening files

feng = open("../Data/Input/LangId.train.English", 'r', encoding='utf8')
ffra = open("../Data/Input/LangId.train.French", 'r', encoding='utf8')
fita = open("../Data/Input/LangId.train.Italian", 'r', encoding='utf8')

eng_string = feng.read(); eng_string = eng_string.lower();
fra_string = ffra.read(); fra_string = fra_string.lower();
ita_string = fita.read(); ita_string = ita_string.lower();

feng.close();
ffra.close();
fita.close();

In [6]:
# cleaning

def clean_text(data):

    # cleaning
    # for i in range(len(data)):
    #     if data[i] == '\'':
    #         continue

    # make lowercase which i think i've already done honestly
    data = data.lower()

    # getting rid of numbers and punctuation
    for char in punctuation:
        if char == '\'': 
            continue
        data = data.replace(char, ' ')
    for num in '1234567890':
        data = data.replace(num, '')

    #whitespace
    data = data.strip()
    data = data.replace("\n", " ")
    re.sub(" +", " ", data)
    data = data.replace("   ", " ")
    data = data.replace("  ", " ")
    # data = " ".join(data.split())
    # data = data.replace(" \'", "\'")
    # data = data.replace("\' ", "\'")

    return data

eng_string = clean_text(eng_string)
ita_string = clean_text(ita_string)
fra_string = clean_text(fra_string)
                               
'''
if i is a space and i+1 is a comma/apostrophe/semicolon/, continue and do nothing else
if i+1 is . and i+2 is \n, change . to </s> and \n to <s> as long as there's a letter after it
    if i is </s> and i+1 is \n, change \n to <s> as long as there's a letter on the next line
if i find "' - s", add '\'',s to the dict and move i forward far enough
THEY USED A DASH TO MARK POSSESSIVES

i might not actually care about punctuation other than apostrophes
    i want english to have a fuckton of 's and s'
    i want french to have a fuckton of j' and t'
    i want spanish to literally have 0 of them
    italian should have l' but so should french
'''
# print(clean_text(eng_string[:2000]))
# print("\n");
# print(clean_text(ita_string[:2000]))
# print("\n");
print(clean_text(fra_string[:2000]))
print("")

approbation du procès verbal de la séance précédente le procès verbal de la séance d ' hier a été distribué y a t il des observations monsieur le président lundi j ' ai soulevé une motion de procédure sur les commentaires de la présidente nicole fontaine que la presse britannique aurait rapportés et qui concernaient sa récente rencontre avec sa majesté la reine elizabeth ii un député travailliste de cette assemblée m miller a répété non pas une fois mais trois fois le lundi le mardi et le mercredi les remarques que la reine aurait faites il a tenté d ' attirer la reine dans une controverse politique et d ' utiliser son nom à des fins politiciennes monsieur le président mme fontaine n ' avait peut être pas connaissance des règles régissant le protocole britannique qui sont en vigueur en la matière mais m miller n ' a aucune excuse il sait pertinemment que notre reine est un chef d ' état apolitique il sait pertinemment que depuis cinquante ans elle a scrupuleusement évité de s ' engager

In [7]:
def make_bigram(data):
    bigram_count = {} # make a dictionary that maps bigrams to their counts

    # CLEANING:

    # data = clean_text(data)
    # data = data.replace("\n", "\n ")
    data = data.split(" ")

    # MAKING nGRAMS
    for i in range(len(data) - 1):
        # skip line breaks
        # if (data[i] == '\n' or data[i+1] == '\n') : continue

        if (data[i], data[i+1]) in bigram_count:            # if it's already counted, increment
            bigram_count[(data[i], data[i + 1])] += 1
        else:
            bigram_count[(data[i], data[i + 1])] = 1        # if it's new, put it in as 1
        
    return bigram_count

def make_unigram(data):
    unigram_count = {} # make a dictionary that maps bigrams to their counts

    # CLEANING:

    # data = clean_text(data)
    # data = data.replace("\n", "\n ")
    data = data.split(" ")

    # MAKING nGRAMS
    for i in data:
        # skip line breaks
        # if (data[i] == '\n' or data[i+1] == '\n') : continue

        if i in unigram_count:            # if it's already counted, increment
            unigram_count[i] += 1
        else:
            unigram_count[i] = 1        # if it's new, put it in as 1


    return unigram_count



# print(make_bigram(fra_string)[0])

In [8]:
from math import log

# scoring a line given its bigrams and unigrams
def score_line(line, bigram, unigram):
    # initials
    score = 0
    n = 1
    V = len(unigram)

    # split by word
    line = line.split(" ")
    # print(line)
    for i in range(len(line) - 1) :
        denom = V 
        numer = n
        # if we've seen it, add its counts
        if line[i] in unigram.keys():
            denom += unigram[line[i]]
        if (line[i], line[i+1]) in bigram.keys():
            numer += bigram[(line[i], line[i+1])]
        
        # divide the counts and take the log
        score += log(numer/denom)
    return score   

In [9]:
ita_bigram = make_bigram(ita_string)
eng_bigram = make_bigram(eng_string)
fra_bigram = make_bigram(fra_string)
ita_unigram = make_unigram(ita_string)
eng_unigram = make_unigram(eng_string)
fra_unigram = make_unigram(fra_string)

# score the whole thing
def score(data) :
    ans = ''
    # scores = {"English" : 0, "Italian" : 0, "French" : 0}
    data = data.split("\n")
    for i in range(len(data)) :
        '''
        compare against all 3 bigrams and unigrams, add the highest score to the string
        '''
        line = data[i]
        line = clean_text(line)
        ans += f'{i + 1} '
        
        scores = {}

        ita_score = score_line(line, ita_bigram, ita_unigram)
        scores[ita_score] = "Italian"

        eng_score = score_line(line, eng_bigram, eng_unigram)
        scores[eng_score] = "English"

        fra_score = score_line(line, fra_bigram, fra_unigram)
        scores[fra_score] = "French"
        
        ans += scores[max(scores.keys())]

        if i < 299: ans += "\n"
        
    return ans

In [10]:
# run the function
def main(test_file) :
    test = open(test_file, "r")
    test_str = test.read()
    test.close()
    ans = score(test_str)
    out = open("../Data/Output/wordLangId.out", "w")
    out.write(ans)
    out.close()
    
main("../Data/Validation/LangId.test")

In [11]:

# compare output and expected
def comp(out_file, sol_file) :
    out = open(out_file, "r")
    sol = open(sol_file, "r")
    num = 0
    out = out.readlines()
    sol = sol.readlines()
    for i in range(len(out)) :
        if (out[i] == sol[i]): num += 1
        else: print(out[i], sol[i])
        
        
    return f'{num}/{len(out)} : {round((num/len(out))*100, 2)}%'

print(comp("../Data/Output/wordLangId.out", "../Data/Validation/labels.sol"))

44 English
 44 Italian

87 French
 87 English

244 English
 244 Italian

248 English
 248 Italian

262 English
 262 Italian

297 English
 297 Italian

294/300 : 98.0%
